In [1]:
from dotenv import load_dotenv

_ = load_dotenv()

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [10]:
from contextlib import ExitStack

# Use contextlib to manually enter the context manager and keep the object
stack = ExitStack()

# Create the memory context manager and enter the context
memory= stack.enter_context(SqliteSaver.from_conn_string(":memory:"))



In [11]:
from uuid import uuid4
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage

"""
In previous examples we've annotated the `messages` state key
with the default `operator.add` or `+` reducer, which always
appends new messages to the end of the existing messages array.

Now, to support replacing existing messages, we annotate the
`messages` key with a customer reducer function, which replaces
messages with the same `id`, and appends them otherwise.
"""
def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    # assign ids to messages that don't have them
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    # merge the new messages with the existing messages
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            # replace any existing messages with the same id
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            # append any new messages to the end
            merged.append(message)
    return merged

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

In [12]:
tool = TavilySearchResults(max_results=2)

In [13]:
class Agent:
    def __init__(self, model, tools, system="", checkpointer=None):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(
            checkpointer=checkpointer,
            interrupt_before=["action"]
        )
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        print(state)
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [14]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-3.5-turbo")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [15]:
messages = [HumanMessage(content="Whats the weather in SF?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in SF?', additional_kwargs={}, response_metadata={}, id='54cedf3a-5fff-47c8-b639-805c2f0e3ee8'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_jL5oUyyfgewT5iL2xRURRPlo', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 152, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BUupQQGuHP6GkStpoeCrUV8wImKxQ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--cf99edd8-bfe1-47d2-8011-e2a83e34aacc-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {

In [9]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', additional_kwargs={}, response_metadata={}, id='05221632-8801-48ec-8ee1-f53ca9c82bea'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_i9oo5Lxg2ckgaKWyMKhA2OC3', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 152, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BUv1KULSLPs3tSJkJUjW5t3XMLp49', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--89f27fe5-7d19-40dd-95cf-7bf3e98cf670-0', tool_calls=[{'name': 'tavily_search_res

In [10]:
abot.graph.get_state(thread).next

('action',)

In [11]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_i9oo5Lxg2ckgaKWyMKhA2OC3', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content="[{'title': 'Thursday, May 8, 2025. San Francisco, CA - Weather Forecast', 'url': 'https://weathershogun.com/weather/usa/ca/san-francisco/480/may/2025-05-08', 'content': 'San Francisco, California Weather: Thursday, May 8, 2025. Cloudy weather, overcast skies with clouds. Day 66°. Night 52°. Precipitation 25 %.', 'score': 0.9639738}, {'title': 'Weather in San Francisco in May 2025 (California)', 'url': 'https://world-weather.info/forecast/usa/san_francisco/may-2025/', 'content': 'Weather in San Francisco in May 2025. San Francisco Weather Forecast for ... Thursday, 8 May. +50°. Day. +61°. Clear sky. Friday, 9 May. +50°. Day. +55°. Clear', 'score': 0.94584644}]", name='tavily_search_results_json', id='9d14dfa4-2dd5-4563-9d2c-66e080944e98', tool_call_id='call_i9oo5Lxg2ckgaKWyMKhA2OC3

In [12]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', additional_kwargs={}, response_metadata={}, id='05221632-8801-48ec-8ee1-f53ca9c82bea'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_i9oo5Lxg2ckgaKWyMKhA2OC3', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 152, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BUv1KULSLPs3tSJkJUjW5t3XMLp49', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--89f27fe5-7d19-40dd-95cf-7bf3e98cf670-0', tool_calls=[{'name': 'tavily_search_res

In [13]:
abot.graph.get_state(thread).next

()

In [14]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)
while abot.graph.get_state(thread).next:
    print("\n", abot.graph.get_state(thread),"\n")
    _input = input("proceed?")
    if _input != "y":
        print("aborting")
        break
    for event in abot.graph.stream(None, thread):
        for v in event.values():
            print(v)

{'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='d271e167-7256-4eb5-887e-707437d02f8c'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_iC6e9ETS2AxYoangMRN3fkFL', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 152, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BUv3QtuE8dkhv0du0rgnnLpNoGDeI', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--a198f7f5-ca10-4739-85b3-fd0aa99c09b2-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'q

proceed? y


Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_iC6e9ETS2AxYoangMRN3fkFL', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'title\': \'Weather in Los Angeles\', \'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Los Angeles\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 34.0522, \'lon\': -118.2428, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1746708959, \'localtime\': \'2025-05-08 05:55\'}, \'current\': {\'last_updated_epoch\': 1746708300, \'last_updated\': \'2025-05-08 05:45\', \'temp_c\': 13.9, \'temp_f\': 57.0, \'is_day\': 0, \'condition\': {\'text\': \'Partly Cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/116.png\', \'code\': 1003}, \'wind_mph\': 2.2, \'wind_kph\': 3.6, \'wind_degree\': 245, \'wind_dir\': \'WSW\', \'pressure_mb\': 1018.0, \'pressure_in\': 30.06, \'precip_mm\': 0.0, \'precip_in\': 0.0